In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/VeRi.zip ./
!unzip VeRi.zip -d ./

Streaming output truncated to the last 5000 lines.
  inflating: ./VeRi/image_train/0665_c012_00062320_0.jpg  
  inflating: ./VeRi/image_train/0665_c012_00062325_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062110_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062115_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062120_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062125_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062130_0.jpg  
  inflating: ./VeRi/image_train/0665_c013_00062135_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063935_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063940_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063945_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063950_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063955_0.jpg  
  inflating: ./VeRi/image_train/0665_c014_00063960_0.jpg  
  inflating: ./VeRi/image_train/0665_c015_00063745_0.jpg  
  inflating: ./VeRi/image_train/0665_c015_00063750_0.jpg  
  inf

In [ ]:
import os, torch
import pandas as pd
from xml.etree import ElementTree as ET
from torch.utils.data import Dataset,DataLoader,random_split
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,root_dir, xml_file,color_file, transform=None ) -> None:
        self.root_dir = root_dir
        self.color_mapping = self.load_color_mapping(color_file)
        self.data = self.parse_xml(xml_file)
        self.transform = transform

    def load_color_mapping(self,color_file):
        color_mapping = {}
        with open(color_file,'r') as file:
            for line in file:
                color_id, color_name = line.strip().split(' ',1)
                color_mapping[int(color_id)-1] = color_name
        return color_mapping

    def parse_xml(self, xml_file):
        data = []
        with open(xml_file,'r') as file:
            tree = ET.fromstring(file.read())
            tree = ET.ElementTree(tree)
            root = tree.getroot()
            for item in root.findall('.//Item'):
                image_name = item.get('imageName')
                color_id = int(item.get('colorID'))
                data.append((image_name,color_id))
        return data

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        img_name, color_id = self.data[idx]
        img_path = os.path.join(self.root_dir, img_name)
        # image = Image.open(img_path).convert("RGB")
        # if self.transform:
        #     image = self.transform(image)
        return img_path, color_id-1, self.color_mapping[color_id-1]




# Main

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/VeRiAttributes.csv")
df.head()

,Unnamed: 0,xmin,ymin,xmax,ymax,color,path
0,0,5.382449,1.217500,123.731544,131.110001,yellow,/content/VeRi/image_train/0001_c001_00016450_0...
1,1,9.657801,0.876281,139.724747,147.047409,yellow,/content/VeRi/image_train/0001_c001_00016460_0...
2,2,3.632896,2.506027,551.147278,330.975647,yellow,/content/VeRi/image_train/0001_c002_00016885_0...
3,3,3.900270,9.683184,394.178070,257.391296,yellow,/content/VeRi/image_train/0001_c002_00016895_0...
4,4,0.651253,1.017353,276.706390,199.564133,yellow,/content/VeRi/image_train/0001_c002_00016905_0...


In [ ]:
train_dataset = CustomDataset("/content/VeRi/image_train","/content/VeRi/train_label.xml","/content/VeRi/list_color.txt")
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)

In [ ]:
len(train_dataset)

37746

In [ ]:
len(df)

17215

In [ ]:
len(train_dataset)-len(df)

20531

# For training only

In [ ]:
unpredicted_paths = []
count = 0
for img_paths, labels, colors in train_loader:
  for i,path in enumerate(img_paths):
    if not (df["path"] == path).any():
      # count += 1
      # print("path not predicted: ", path, count)
      unpredicted_paths.append({"path":path, "label": labels[i], "color": colors[i]})

In [ ]:
len(unpredicted_paths)

20531

In [ ]:
class UnpredictedPathsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        path = item["path"]
        label = item["label"]
        color = item["color"]
        return path, label, color

In [ ]:
train_dataset = UnpredictedPathsDataset(unpredicted_paths)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)

# Prediction Part

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')
model = model.to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 206.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.5s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-2-16 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 89.3M/89.3M [00:00<00:00, 376MB/s]

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


In [ ]:
predicted_data = []
count = 0
for img_paths, labels, colors in train_loader:
  results = model(img_paths)

  for i,df in enumerate(results.pandas().xywhn):
    # tdf = df[df["name"] == "car"]
    tdf = df[df["name"].str.contains("car|truck|bus|train|suitcase|cell phone|parking meter", case=False)]

    if len(tdf) > 0:
      item = tdf.iloc[0]
      p_data = {"xcenter":item["xcenter"], "ycenter":item["ycenter"], "width":item["width"], "height":item["height"], "color":colors[i], "path":img_paths[i], "name": item["name"]}
      predicted_data.append(p_data)

  count += 1
  print("Batch count: ", count)
  break

Batch count:  1


In [ ]:
results.panda()

106

In [ ]:
results.pandas().xywhn[0]

,xcenter,ycenter,width,height,confidence,class,name
0,0.506070,0.487967,0.933439,0.935112,0.327506,12,parking meter
1,0.498372,0.505172,0.923651,0.955844,0.281676,7,truck


In [ ]:
# bus, truck, car, train, suitcase, cell phone, parknig meter

In [ ]:
def prediction_start(dataloader):
  predicted_data = []
  count = 0
  for img_paths, labels, colors in dataloader:
    results = model(img_paths)

    for i,df in enumerate(results.pandas().xywhn):
      # tdf = df[df["name"] == "car"]
      tdf = df[df["name"].str.contains("car|truck|bus|train|suitcase|cell phone|parking meter", case=False)]

      if len(tdf) > 0:
        item = tdf.iloc[0]
        p_data = {"xcenter":item["xcenter"], "ycenter":item["ycenter"], "width":item["width"], "height":item["height"], "color":colors[i], "path":img_paths[i], "name": item["name"]}
        predicted_data.append(p_data)

    count += 1
    print("Batch count: ", count)
  return predicted_data

In [ ]:
predicted_data_train = prediction_start(train_loader)
df = pd.DataFrame(predicted_data_train)
df.to_csv("/content/drive/MyDrive/VeRiAttributesTrainAll.csv")

Batch count:  1
Batch count:  2
Batch count:  3
Batch count:  4
Batch count:  5
Batch count:  6
Batch count:  7
Batch count:  8
Batch count:  9
Batch count:  10
Batch count:  11
Batch count:  12
Batch count:  13
Batch count:  14
Batch count:  15
Batch count:  16
Batch count:  17
Batch count:  18
Batch count:  19
Batch count:  20
Batch count:  21
Batch count:  22
Batch count:  23
Batch count:  24
Batch count:  25
Batch count:  26
Batch count:  27
Batch count:  28
Batch count:  29
Batch count:  30
Batch count:  31
Batch count:  32
Batch count:  33
Batch count:  34
Batch count:  35
Batch count:  36
Batch count:  37
Batch count:  38
Batch count:  39
Batch count:  40
Batch count:  41
Batch count:  42
Batch count:  43
Batch count:  44
Batch count:  45
Batch count:  46
Batch count:  47
Batch count:  48
Batch count:  49
Batch count:  50
Batch count:  51
Batch count:  52
Batch count:  53
Batch count:  54
Batch count:  55
Batch count:  56
Batch count:  57
Batch count:  58
Batch count:  59
Batch 

In [ ]:
len(df)

34761

In [ ]:
df["path"].nunique()

34761

In [ ]:
df.head()

,xcenter,ycenter,width,height,color,path,name
0,0.139106,0.150256,0.218042,0.297020,yellow,/content/VeRi/image_train/0001_c001_00016450_0...,parking meter
1,0.516875,0.484234,0.891260,0.926835,yellow,/content/VeRi/image_train/0001_c001_00016460_0...,truck
2,0.506070,0.487967,0.933439,0.935112,yellow,/content/VeRi/image_train/0001_c001_00016470_0...,parking meter
3,0.505929,0.506680,0.954538,0.959184,yellow,/content/VeRi/image_train/0001_c001_00016480_0...,truck
4,0.499871,0.505723,0.959701,0.972941,yellow,/content/VeRi/image_train/0001_c001_00016490_0...,truck
